https://towardsdatascience.com/text-analysis-basics-in-python-443282942ec5    

https://towardsdatascience.com/topic-modeling-and-latent-dirichlet-allocation-in-python-9bf156893c24

https://medium.com/@yanlinc/how-to-build-a-lda-topic-model-using-from-text-601cdcbfd3a6

https://methods.sagepub.com/base/download/DatasetHowToGuide/latent-dirichlet-allocation-in-news-2016-python

In [11]:
import pandas as pd
#df = pd.read_csv('C:/Users/jeanl/Desktop/trial.csv',index_col=False)
from textblob import TextBlob



In [12]:
df = pd.read_pickle("C:/Users/mwamb/OneDrive/Desktop/tenthdimensionanalytics/WFP/processed_data/wfpdata_clean")
df

,cluster,words,no_contract,words_str,tokenized,lower,no_punc,stopwords_removed,pos_tags,wordnet_pos,lemmatized
0,WFP,FOOD SECURITY \n AND NUTRITION \nIN THE WORLD\...,"[FOOD, SECURITY, AND, NUTRITION, IN, THE, WORL...",FOOD SECURITY AND NUTRITION IN THE WORLD THE S...,"[FOOD, SECURITY, AND, NUTRITION, IN, THE, WORL...","[food, security, and, nutrition, in, the, worl...","[food, security, and, nutrition, in, the, worl...","[food, security, nutrition, world, state, safe...","[(food, NN), (security, NN), (nutrition, NN), ...","[(food, n), (security, n), (nutrition, n), (wo...","[food, security, nutrition, world, state, safe..."


In [3]:
df['polarity'] = df['words'].apply(lambda x: TextBlob(x).polarity)
df['subjective'] = df['words'].apply(lambda x: TextBlob(x).subjectivity)

In [4]:
from nltk.corpus import stopwords
stoplist = stopwords.words('english') + ['though','www','https']

In [5]:
from sklearn.feature_extraction.text import CountVectorizer
c_vec = CountVectorizer(stop_words=stoplist, ngram_range=(2,3))


In [6]:
# matrix of ngrams
ngrams = c_vec.fit_transform(df['words'])
# count frequency of ngrams
count_values = ngrams.toarray().sum(axis=0)


In [7]:
# list of ngrams
vocab = c_vec.vocabulary_
df_ngram = pd.DataFrame(sorted([(count_values[i],k) for k,i in vocab.items()], reverse=True)).rename(columns={0: 'frequency', 1:'bigram/trigram'})

In [8]:
df_ngram

,frequency,bigram/trigram
0,949,food insecurity
1,615,food security
2,433,acute food
3,392,acute food insecurity
4,368,food crises
...,...,...
227313,1,00 11 51
227314,1,00 11
227315,1,00 10 3n
227316,1,00 10 1swedenn


In [9]:
df_ngram['polarity'] = df_ngram['bigram/trigram'].apply(lambda x: TextBlob(x).polarity)
df_ngram['subjective'] = df_ngram['bigram/trigram'].apply(lambda x: TextBlob(x).subjectivity)

In [10]:
df_ngram['subjective']

0         0.0
1         0.0
2         0.9
3         0.9
4         0.0
         ... 
227313    0.0
227314    0.0
227315    0.0
227316    0.0
227317    0.0
Name: subjective, Length: 227318, dtype: float64

## Topic modeling

NMF models
Non-Negative Matrix Factorization (NMF) is a matrix decomposition method, which decomposes a matrix into the product of W and H of non-negative elements. The default method optimizes the distance between the original matrix and WH, i.e., the Frobenium norm. Below is an example where we use NMF to produce 3 topics and we showed 3 bigrams/trigrams in each topic.

Source: https://scikit-learn.org/stable/auto_examples/applications/plot_topics_extraction_with_nmf_lda.html

In [11]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import NMF
from sklearn.pipeline import make_pipeline

In [3]:
tfidf_vectorizer = TfidfVectorizer(stop_words=stoplist, ngram_range=(2,3))
nmf = NMF(n_components=10)
pipe = make_pipeline(tfidf_vectorizer, nmf)
pipe.fit(df['words'])

NameError: name 'TfidfVectorizer' is not defined

In [1]:
def print_top_words(model, feature_names, n_top_words):
    for topic_idx, topic in enumerate(model.components_):
        message = "Topic %d: " % topic_idx
        message += ", ".join([feature_names[i]
                             for i in topic.argsort()[:-n_top_words - 1:-1]])
        
        
        print(message)
        
    print()


In [2]:
print_top_words(nmf, tfidf_vectorizer.get_feature_names(), n_top_words=7)


NameError: name 'nmf' is not defined

LDA models

   
Latent Dirichlet Allocation is a generative probabilistic model for collections of discrete dataset such as text corpora. It is also a topic model that is used for discovering abstract topics from a collection of documents.

Source: https://scikit-learn.org/stable/auto_examples/applications/plot_topics_extraction_with_nmf_lda.html

In [15]:

from sklearn.decomposition import LatentDirichletAllocation
tfidf_vectorizer = TfidfVectorizer(stop_words=stoplist, ngram_range=(2,3))
lda = LatentDirichletAllocation(n_components=3)
pipe = make_pipeline(tfidf_vectorizer, lda)
pipe.fit(df['words'])


Pipeline(memory=None,
         steps=[('tfidfvectorizer',
                 TfidfVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.float64'>,
                                 encoding='utf-8', input='content',
                                 lowercase=True, max_df=1.0, max_features=None,
                                 min_df=1, ngram_range=(2, 3), norm='l2',
                                 preprocessor=None, smooth_idf=True,
                                 stop_words=['i', 'me', 'my', 'myself', 'we',
                                             'our', 'ours', 'ourse...
                ('latentdirichletallocation',
                 LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
                                           evaluate_every=-1,
                                           learning_decay=0.7,
                                           learning_method='batch',

In [16]:
def print_top_words(model, feature_names, n_top_words):
    for topic_idx, topic in enumerate(model.components_):
        message = "Topic #%d: " % topic_idx
        message += ", ".join([feature_names[i]
                             for i in topic.argsort()[:-n_top_words - 1:-1]])
        print(message)
    print()


In [17]:
print_top_words(lda, tfidf_vectorizer.get_feature_names(), n_top_words=5)

Topic #0: food security, food insecurity, acute food, acute food insecurity, food crises
Topic #1: food security, food insecurity, acute food, acute food insecurity, food crises
Topic #2: food insecurity, food security, acute food, acute food insecurity, food crises



## similarity 

In [1]:
import spacy 
  


In [2]:
nlp = spacy.load('en_core_web_md') 


In [25]:
words = "player players football game match david tenis 'df'"

In [26]:
tokens = nlp(words) 

In [27]:
for token in tokens: 
    # Printing the following attributes of each token. 
    # text: the word string, has_vector: if it contains 
    # a vector representation in the model,  
    # vector_norm: the algebraic norm of the vector, 
    # is_oov: if the word is out of vocabulary. 
    print(token.text, token.has_vector, token.vector_norm, token.is_oov)

player True 7.1004224 False
players True 7.18582 False
football True 7.4812903 False
game True 7.441479 False
match True 6.2072406 False
david True 6.5919833 False
tenis True 7.365459 False
' True 6.137929 False
df True 6.2081633 False
' True 6.137929 False


In [36]:


print("Similarity:", tokens.similarity(tokens)) 

Similarity: <built-in method similarity of spacy.tokens.doc.Doc object at 0x0000021A55567F48>


In [40]:
doc1 = nlp(" i love football")
doc2 = nlp(" football game is nie")



In [41]:
print(doc1.similarity(doc2))

0.7299463936488044


In [42]:
doc = nlp("we love our parents")

token1=doc[3]
token2= doc[1]

print(token1.similarity(token2))

0.40540856


In [43]:
doc = nlp("let's start the game ")
token = nlp("play")[0]

In [44]:
print(doc.similarity(token))

0.6587394569716348


In [45]:
text1 = nlp("let's meet at java and discuss more")[2:5]
text2 = nlp(" we ca have our meeting at a restaurant")

print(text1.similarity(text2))

0.6792010226689111


In [47]:
print(text1[4].vector)

[-2.3336e-01  2.5781e-01 -2.0162e-01  5.2606e-01 -2.8509e-02 -1.1073e-01
  1.2757e-02 -2.0570e-01 -2.1759e-01  2.9903e+00 -3.4121e-01 -3.4941e-01
  1.5605e-01  3.1501e-03  4.7926e-02  6.0154e-02 -2.5585e-01  5.3310e-01
 -5.2395e-01 -2.0549e-01  3.9880e-01  9.4371e-02 -8.1085e-03 -1.4268e-01
  6.9565e-02  1.9760e-01  3.3227e-01  2.6695e-01  1.2351e-01 -2.4241e-01
  3.0009e-02  1.3260e-01  1.9259e-01  2.2053e-01  2.2206e-01  4.1598e-01
  4.7964e-01  1.1994e-01 -4.0417e-01 -2.7630e-01 -3.4020e-01 -3.0179e-01
 -6.7554e-02  1.0889e-01 -3.3474e-01  3.0955e-01  3.2112e-02  2.5001e-01
  1.2119e-01 -2.1610e-01  2.8064e-01  1.6946e-01 -2.1553e-01 -5.8009e-02
 -5.2865e-02  5.2013e-02 -4.4257e-01 -2.5601e-01 -8.0894e-02  1.9187e-01
  3.5572e-01  2.6406e-01 -4.4727e-02  4.0338e-01  2.8569e-01 -2.7695e-01
  1.7033e-01  1.4237e-01  1.0442e-01 -7.8184e-02  2.5931e-01 -1.5034e-01
  2.0141e-01 -2.1194e-01  2.6104e-02  2.4404e-01 -4.0048e-02 -3.8523e-01
  1.8870e-01  1.3488e-01  2.8087e-01 -2.4757e-01 -3

In [48]:
 
words = "we love our lives" 
  
tokens = nlp(words) 
  
for token in tokens: 
    # Printing the following attributes of each token. 
    # text: the word string, has_vector: if it contains 
    # a vector representation in the model,  
    # vector_norm: the algebraic norm of the vector, 
    # is_oov: if the word is out of vocabulary. 
    print(token.text, token.has_vector, token.vector_norm, token.is_oov) 
  
token1, token2 = tokens[0], tokens[1] 
  
print("Similarity:", token1.similarity(token2))

we True 5.260865 False
love True 6.04035 False
our True 5.592738 False
lives True 6.036322 False
Similarity: 0.5461829


In [ ]:
# Topic modeling with bigram 

### NMF models Non-Negative Matrix Factorization (NMF) 

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import NMF
from sklearn.pipeline import make_pipeline

tfidf_vectorizer = TfidfVectorizer(ngram_range=(2,3))
nmf = NMF(n_components=5)
pipe = make_pipeline(tfidf_vectorizer, nmf)
pipe.fit(nouns)


def print_top_words(model, feature_names, n_top_words):
    for topic_idx, topic in enumerate(model.components_):
        message = "Topic %d: " % topic_idx
        message += ", ".join([feature_names[i]
                             for i in topic.argsort()[:-n_top_words - 1:-1]])
        
        
        print(message)
        
    print()


print_top_words(nmf, tfidf_vectorizer.get_feature_names(), n_top_words=7)


## Topic extraction with Non-negative Matrix Factorization and Latent Dirichlet Allocation


from sklearn.decomposition import LatentDirichletAllocation
tfidf_vectorizer = TfidfVectorizer(ngram_range=(2,3))
lda = LatentDirichletAllocation(n_components=3)
pipe = make_pipeline(tfidf_vectorizer, lda)
pipe.fit(nouns)

def print_top_words(model, feature_names, n_top_words):
    for topic_idx, topic in enumerate(model.components_):
        words = "Topic %d: " % topic_idx
        words += ", ".join([feature_names[i]
                             for i in topic.argsort()[:-n_top_words - 1:-1]])
        print(words)
    print()

print_top_words(lda, tfidf_vectorizer.get_feature_names(), n_top_words=6)

